In [1]:
# Amazon Repatriation Automation
## Jamie needs to concatenate sales data received from markets into one CSV file
#### Concatenate ICLA and Classical Sheet into one CSV
##### Jamie needs to send over Cost Feed from Amazon, Concatenated Sales Feed from markets, ICLA exceptions / Classical Sheet and International Catalogue
### remove all currency symbols from cost data
###### create dashboard from the data

In [2]:
#importing libraries

import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from google.cloud import bigquery


In [3]:
#importing sales data

sales = pd.read_csv('G:\Amazon Repatriation May 2020\Amazon Repat - Amazon Data April.csv', dtype={'ean': str})

In [4]:
#importing cost data

costs = pd.read_csv('G:\Amazon Repatriation May 2020\Amazon Cost Feed Combined - April.csv', dtype={'UPC': np.int64 })

In [5]:
### reduce table size 
costs2 = costs[['Region','UPC','Transfer Price','Dealer Price','Net Unit Price','Currency ']]

In [6]:
costs2

,Region,UPC,Transfer Price,Dealer Price,Net Unit Price,Currency
0,DE,8810181520,0.617,5.35,4.92,EUR
1,DE,8810255023,0.737,3.88,3.24,EUR
2,DE,8810328529,0.617,3.88,3.24,EUR
3,DE,8810334124,0.628,4.85,4.49,EUR
4,DE,8810603329,0.602,3.88,3.24,EUR
...,...,...,...,...,...,...
83526,UK,5060000000000,1.000,8.15,6.76,GBP
83527,UK,5060000000000,1.000,8.35,6.93,GBP
83528,UK,603000000000,1.000,79.99,66.39,GBP
83529,UK,5060000000000,1.000,7.59,6.30,GBP


In [7]:
### amount of rows in cost data
costs2.shape

(83531, 6)

In [8]:
#keeping any rows with Transfer Price below .30 euros
costs3 = costs2[costs2['Transfer Price'] >= .3] 

In [9]:
### amount of rows in cost data
costs3.shape

(55654, 6)

In [10]:
###check for duplicates rows
sum(costs3.duplicated())

18904

In [11]:
##drop duplicates
costs4 = costs3.drop_duplicates(ignore_index = True)

In [12]:
###check for duplicates rows
sum(costs4.duplicated())

0

In [13]:
#show imported costs data

costs4

,Region,UPC,Transfer Price,Dealer Price,Net Unit Price,Currency
0,DE,8810181520,0.617,5.35,4.92,EUR
1,DE,8810255023,0.737,3.88,3.24,EUR
2,DE,8810328529,0.617,3.88,3.24,EUR
3,DE,8810334124,0.628,4.85,4.49,EUR
4,DE,8810603329,0.602,3.88,3.24,EUR
...,...,...,...,...,...,...
36745,UK,194000000000,1.000,13.75,11.41,GBP
36746,UK,5060000000000,3.000,7.59,6.30,GBP
36747,UK,5060000000000,1.000,4.70,3.90,GBP
36748,UK,9330000000000,1.000,13.99,11.61,GBP


In [14]:
#show imported sales data

sales

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,ean,artist_primary,item_name,unpacked_units,Report Type
0,2020,4,28/04/2020,DE,UMG DE,42282998620,Magnum,Vigilante,2,Gross Shipments
1,2020,4,29/04/2020,DE,UMG DE,602527000000,Element of Crime,Immer da wo du bist bin ich nie [Import allemand],1,Gross Shipments
2,2020,4,21/04/2020,DE,UMG DE,28947919322,The English Concert Trevor Pinnock,Sämtliche Orchesteraufnahmen,1,Gross Shipments
3,2020,4,25/04/2020,ES,UMG ES,602557000000,Van Morrison,Keep Me Singing [Vinyl LP],1,Gross Shipments
4,2020,4,25/04/2020,DE,UMG DE,602499000000,Metallica,St.Anger [Vinyl LP],1,Gross Shipments
...,...,...,...,...,...,...,...,...,...,...
341899,2020,4,22/04/2020,www.amazon.de,UMG DE,602578000000,The Avett Brothers,The Avett Brothers - Closer Than Together,1,Monthly Sales
341900,2020,4,20/04/2020,www.amazon.es,UMG ES,602528000000,David Bustamante,Sus 50 Mejores Canciones [Import USA],1,Monthly Sales
341901,2020,4,09/04/2020,www.amazon.es,UMG DE,28947939696,Villaume,Tchaïkovski: Iolanta,1,Monthly Sales
341902,2020,4,18/04/2020,www.amazon.co.uk,UMG UK,731454000000,"Evans, Bill",Bill Evans At The Montreux Jazz Festival,1,Monthly Sales


In [15]:
#concatenating inventrory owner groups, vendor groups for streamlined data

sales.loc[sales['inventory_owner_group'] == 'UK', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.co.uk', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'DE', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.de', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'IT', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.it', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'FR', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.fr', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'ES', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.es', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'Pan-EU SCO', 'inventory_owner_group_combined'] = 'Pan-EU SCO'
sales.loc[sales['vendor_group'] == 'UMG UK', 'vendor_group2'] = 'UK'
sales.loc[sales['vendor_group'] == 'UMG DE', 'vendor_group2'] = 'DE'
sales.loc[sales['vendor_group'] == 'UMG FR', 'vendor_group2'] = 'FR'
sales.loc[sales['vendor_group'] == 'UMG IT', 'vendor_group2']  ='IT'
sales.loc[sales['vendor_group'] == 'UMG ES', 'vendor_group2'] ='ES'
sales.loc[sales['vendor_group'] == 'UMG AT', 'vendor_group2'] ='AT'
sales.loc[sales['vendor_group'] == '3rd Party', 'vendor_group2'] = '3rd Party'

In [16]:
#creating movement key

sales['movement'] = sales['vendor_group'] + ' to ' +sales['inventory_owner_group_combined'] 

In [17]:
#show new table

sales.head()

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,ean,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,vendor_group2,movement
0,2020,4,28/04/2020,DE,UMG DE,42282998620,Magnum,Vigilante,2,Gross Shipments,DE,DE,UMG DE to DE
1,2020,4,29/04/2020,DE,UMG DE,602527000000,Element of Crime,Immer da wo du bist bin ich nie [Import allemand],1,Gross Shipments,DE,DE,UMG DE to DE
2,2020,4,21/04/2020,DE,UMG DE,28947919322,The English Concert Trevor Pinnock,Sämtliche Orchesteraufnahmen,1,Gross Shipments,DE,DE,UMG DE to DE
3,2020,4,25/04/2020,ES,UMG ES,602557000000,Van Morrison,Keep Me Singing [Vinyl LP],1,Gross Shipments,ES,ES,UMG ES to ES
4,2020,4,25/04/2020,DE,UMG DE,602499000000,Metallica,St.Anger [Vinyl LP],1,Gross Shipments,DE,DE,UMG DE to DE


In [18]:
sales

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,ean,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,vendor_group2,movement
0,2020,4,28/04/2020,DE,UMG DE,42282998620,Magnum,Vigilante,2,Gross Shipments,DE,DE,UMG DE to DE
1,2020,4,29/04/2020,DE,UMG DE,602527000000,Element of Crime,Immer da wo du bist bin ich nie [Import allemand],1,Gross Shipments,DE,DE,UMG DE to DE
2,2020,4,21/04/2020,DE,UMG DE,28947919322,The English Concert Trevor Pinnock,Sämtliche Orchesteraufnahmen,1,Gross Shipments,DE,DE,UMG DE to DE
3,2020,4,25/04/2020,ES,UMG ES,602557000000,Van Morrison,Keep Me Singing [Vinyl LP],1,Gross Shipments,ES,ES,UMG ES to ES
4,2020,4,25/04/2020,DE,UMG DE,602499000000,Metallica,St.Anger [Vinyl LP],1,Gross Shipments,DE,DE,UMG DE to DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
341899,2020,4,22/04/2020,www.amazon.de,UMG DE,602578000000,The Avett Brothers,The Avett Brothers - Closer Than Together,1,Monthly Sales,DE,DE,UMG DE to DE
341900,2020,4,20/04/2020,www.amazon.es,UMG ES,602528000000,David Bustamante,Sus 50 Mejores Canciones [Import USA],1,Monthly Sales,ES,ES,UMG ES to ES
341901,2020,4,09/04/2020,www.amazon.es,UMG DE,28947939696,Villaume,Tchaïkovski: Iolanta,1,Monthly Sales,ES,DE,UMG DE to ES
341902,2020,4,18/04/2020,www.amazon.co.uk,UMG UK,731454000000,"Evans, Bill",Bill Evans At The Montreux Jazz Festival,1,Monthly Sales,UK,UK,UMG UK to UK


In [19]:
#splitting up strings of UPC Amazon delivers to us

new = sales['ean'].str.split("/", n =-1, expand = True) 
sales["ean1"]= new[0] 
sales["ean2"]= new[1] 
sales["ean3"]= new[2] 
sales["ean4"]= new[3] 
sales["ean5"]= new[4] 
sales.drop(columns =["ean"], inplace = True) 
sales

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,vendor_group2,movement,ean1,ean2,ean3,ean4,ean5
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,DE,UMG DE to DE,42282998620,None,None,None,None
1,2020,4,29/04/2020,DE,UMG DE,Element of Crime,Immer da wo du bist bin ich nie [Import allemand],1,Gross Shipments,DE,DE,UMG DE to DE,602527000000,None,None,None,None
2,2020,4,21/04/2020,DE,UMG DE,The English Concert Trevor Pinnock,Sämtliche Orchesteraufnahmen,1,Gross Shipments,DE,DE,UMG DE to DE,28947919322,None,None,None,None
3,2020,4,25/04/2020,ES,UMG ES,Van Morrison,Keep Me Singing [Vinyl LP],1,Gross Shipments,ES,ES,UMG ES to ES,602557000000,None,None,None,None
4,2020,4,25/04/2020,DE,UMG DE,Metallica,St.Anger [Vinyl LP],1,Gross Shipments,DE,DE,UMG DE to DE,602499000000,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341899,2020,4,22/04/2020,www.amazon.de,UMG DE,The Avett Brothers,The Avett Brothers - Closer Than Together,1,Monthly Sales,DE,DE,UMG DE to DE,602578000000,None,None,None,None
341900,2020,4,20/04/2020,www.amazon.es,UMG ES,David Bustamante,Sus 50 Mejores Canciones [Import USA],1,Monthly Sales,ES,ES,UMG ES to ES,602528000000,None,None,None,None
341901,2020,4,09/04/2020,www.amazon.es,UMG DE,Villaume,Tchaïkovski: Iolanta,1,Monthly Sales,ES,DE,UMG DE to ES,28947939696,None,None,None,None
341902,2020,4,18/04/2020,www.amazon.co.uk,UMG UK,"Evans, Bill",Bill Evans At The Montreux Jazz Festival,1,Monthly Sales,UK,UK,UMG UK to UK,731454000000,None,None,None,None


In [21]:
sales3 = sales.fillna(0)


In [22]:
##change ean columns to int to match up with costs data

sales3['eanone']= sales3['ean1'].astype(float).astype(np.int64)
sales3['eantwo']= sales3['ean2'].astype(float).astype(np.int64)
sales3['eanthree']= sales3['ean3'].astype(float).astype(np.int64)
sales3['eanfour']= sales3['ean4'].astype(float).astype(np.int64)
sales3['eanfive']= sales3['ean5'].astype(float).astype(np.int64)


In [23]:
### drop rows that are redundant 
del sales3['ean1']
del sales3['ean2']
del sales3['ean3']
del sales3['ean4']
del sales3['ean5']

In [24]:
sales3

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,vendor_group2,movement,eanone,eantwo,eanthree,eanfour,eanfive
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,DE,UMG DE to DE,42282998620,0,0,0,0
1,2020,4,29/04/2020,DE,UMG DE,Element of Crime,Immer da wo du bist bin ich nie [Import allemand],1,Gross Shipments,DE,DE,UMG DE to DE,602527000000,0,0,0,0
2,2020,4,21/04/2020,DE,UMG DE,The English Concert Trevor Pinnock,Sämtliche Orchesteraufnahmen,1,Gross Shipments,DE,DE,UMG DE to DE,28947919322,0,0,0,0
3,2020,4,25/04/2020,ES,UMG ES,Van Morrison,Keep Me Singing [Vinyl LP],1,Gross Shipments,ES,ES,UMG ES to ES,602557000000,0,0,0,0
4,2020,4,25/04/2020,DE,UMG DE,Metallica,St.Anger [Vinyl LP],1,Gross Shipments,DE,DE,UMG DE to DE,602499000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341899,2020,4,22/04/2020,www.amazon.de,UMG DE,The Avett Brothers,The Avett Brothers - Closer Than Together,1,Monthly Sales,DE,DE,UMG DE to DE,602578000000,0,0,0,0
341900,2020,4,20/04/2020,www.amazon.es,UMG ES,David Bustamante,Sus 50 Mejores Canciones [Import USA],1,Monthly Sales,ES,ES,UMG ES to ES,602528000000,0,0,0,0
341901,2020,4,09/04/2020,www.amazon.es,UMG DE,Villaume,Tchaïkovski: Iolanta,1,Monthly Sales,ES,DE,UMG DE to ES,28947939696,0,0,0,0
341902,2020,4,18/04/2020,www.amazon.co.uk,UMG UK,"Evans, Bill",Bill Evans At The Montreux Jazz Festival,1,Monthly Sales,UK,UK,UMG UK to UK,731454000000,0,0,0,0


In [25]:
### merge data
combo = sales3.merge(costs4, how='inner', left_on=["eanone"or"eantwo"or"eanthree"or"eanfour"or"eanfive","vendor_group2"], right_on=["UPC","Region"])

In [26]:
combo

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,eantwo,eanthree,eanfour,eanfive,Region,UPC,Transfer Price,Dealer Price,Net Unit Price,Currency
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0,0,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0,0,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0,0,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0,0,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,0,0,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179974,2020,4,04/04/2020,www.amazon.de,UMG FR,Rudolf Serkin,The Originals-Brahms: Die Cellosonaten,1,Monthly Sales,DE,...,0,0,0,0,FR,28947937678,0.5038,4.89,4.45479,EUR
179975,2020,4,22/04/2020,www.amazon.co.uk,UMG UK,Val Doonican - His Special Years Box Set (Musi...,Very Best: His Special Years,1,Monthly Sales,UK,...,0,0,0,0,UK,28946640623,0.7700,7.55,6.19000,GBP
179976,2020,4,21/04/2020,www.amazon.de,UMG UK,Various Artists,Various Artists - La Phil Centenary Edition,1,Monthly Sales,DE,...,0,0,0,0,UK,28948361007,15.8700,59.47,48.77000,GBP
179977,2020,4,19/04/2020,www.amazon.co.uk,UMG IT,Wolfram Christ,Trout Quintet & Death And The Maiden (La Trota),1,Monthly Sales,UK,...,0,0,0,0,IT,28947842224,0.5800,3.00,2.44340,EUR


In [27]:
###amount of rows
combo.shape

(179979, 23)

In [28]:
### drop row which didn't match
combo4 = combo[combo['UPC'].notna()]

In [29]:
### amount of rows
combo4.shape

(179979, 23)

In [30]:
###check for duplicates rows
sum(combo4.duplicated())

3893

In [31]:
#importing international catalogue data

international_catalogue = pd.read_csv(r'G:\Amazon Repatriation May 2020\UMGI Physical Catalogue_17042020.csv', dtype={'UPC': float})

In [32]:
#showing international catalogue data

international_catalogue

,Product ID,Releasing Territory,Product Type,Marketing Div,Artist,Title,Format Group,Format/Config Code,Release Date,UPC,...,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47
0,605.0,International,Commercial,Frontline,Dalida,Forever Dalida,CD,CD-A,16/02/2004,6.024980e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,607.0,International,Commercial,Frontline,George Strait,50 Number Ones,CD,2CD-A,04/10/2004,8.817039e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,638.0,International,Commercial,Frontline,Guns N' Roses,Welcome To The Videos,DVD,DVDV-A,15/12/2003,6.024990e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,639.0,International,Commercial,Frontline,Guns N' Roses,Use Your Illusion I,DVD,DVDV-A,12/01/2004,6.024990e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,640.0,International,Commercial,Frontline,Guns N' Roses,Use Your Illusion II,DVD,DVDV-A,12/01/2004,6.024990e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21817,1436035.0,International,Commercial,Frontline,Dan Croll,Grand Plan,Vinyl,LP,21/08/2020,5.060500e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21818,1436036.0,International,Commercial,Frontline,Dan Croll,Grand Plan,CD,CD-A,21/08/2020,5.060500e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21819,1436040.0,France,Commercial,Frontline,Florent PAGNY,Savoir aimer,Vinyl,LP,21/08/2020,6.007540e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21820,1436052.0,International,Commercial,Frontline,John Lee Hooker,Live At Montreux 1983 / 1990,Vinyl,2LP,25/09/2020,6.025090e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
### drop redundant columns
cat2 = international_catalogue[['Rep Owner','UPC']]

In [34]:
## show columns
cat2

,Rep Owner,UPC
0,Barclay,6.024980e+11
1,MCA Nashville,8.817039e+09
2,Geffen,6.024990e+11
3,Geffen,6.024990e+11
4,Geffen,6.024990e+11
...,...,...
21817,Caroline International,5.060500e+12
21818,Caroline International,5.060500e+12
21819,FR Capitol,6.007540e+11
21820,Eagle Rock Entertainment Ltd,6.025090e+11


In [35]:
##drop duplicates
cat3 = cat2.drop_duplicates(ignore_index = True)

In [36]:
###check for duplicates rows
sum(cat3.duplicated())

0

In [37]:
## show cat3
cat3

,Rep Owner,UPC
0,Barclay,6.024980e+11
1,MCA Nashville,8.817039e+09
2,Geffen,6.024990e+11
3,USM/Universal (UMGI),6.024980e+11
4,Verve US,6.024990e+11
...,...,...
7995,Deutsche Grammophon Intl,2.894839e+10
7996,Deutsche Grammophon Intl,2.894838e+10
7997,Deutsche Grammophon Intl,2.894839e+10
7998,DE CJ/Jazz,6.025070e+11


In [38]:
##only keep first label
cat4 = cat3.drop_duplicates(subset='UPC', keep="first")

In [39]:
#show cat4
cat4

,Rep Owner,UPC
0,Barclay,6.024980e+11
1,MCA Nashville,8.817039e+09
2,Geffen,6.024990e+11
9,UMTV Intl,4.400686e+10
10,Emarcy,4.228388e+10
...,...,...
7994,Deutsche Grammophon Intl,2.894839e+10
7995,Deutsche Grammophon Intl,2.894839e+10
7996,Deutsche Grammophon Intl,2.894838e+10
7997,Deutsche Grammophon Intl,2.894839e+10


In [40]:
### drop row which didn't match
cat5 = cat4[cat4['UPC'].notna()]

In [41]:
cat6 = cat5['UPC'].astype(float).astype(np.int64)

In [42]:
#merging our dataset and international catalogue to return Rep Owner for each UPC

combo_cat = pd.merge(combo4,cat4[['UPC','Rep Owner']],
                     left_on="eanone"or"eantwo"or"eanthree"or"eanfour"or"eanfive", right_on=['UPC'], how='left')

In [43]:
#showing new data

combo_cat

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,eanfour,eanfive,Region,UPC_x,Transfer Price,Dealer Price,Net Unit Price,Currency,UPC_y,Rep Owner
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179974,2020,4,04/04/2020,www.amazon.de,UMG FR,Rudolf Serkin,The Originals-Brahms: Die Cellosonaten,1,Monthly Sales,DE,...,0,0,FR,28947937678,0.5038,4.89,4.45479,EUR,2.894794e+10,Deutsche Grammophon Intl
179975,2020,4,22/04/2020,www.amazon.co.uk,UMG UK,Val Doonican - His Special Years Box Set (Musi...,Very Best: His Special Years,1,Monthly Sales,UK,...,0,0,UK,28946640623,0.7700,7.55,6.19000,GBP,2.894664e+10,Decca
179976,2020,4,21/04/2020,www.amazon.de,UMG UK,Various Artists,Various Artists - La Phil Centenary Edition,1,Monthly Sales,DE,...,0,0,UK,28948361007,15.8700,59.47,48.77000,GBP,2.894836e+10,Deutsche Grammophon Intl
179977,2020,4,19/04/2020,www.amazon.co.uk,UMG IT,Wolfram Christ,Trout Quintet & Death And The Maiden (La Trota),1,Monthly Sales,UK,...,0,0,IT,28947842224,0.5800,3.00,2.44340,EUR,2.894784e+10,Decca


In [44]:
### count rows and columsn in combo2
combo_cat.shape

(179979, 25)

In [45]:
###drop caroline labels
combo_cat2 = combo_cat[combo_cat['Rep Owner'] != 'Caroline/Astralwerks']

In [46]:
### check row length 
combo_cat2.shape

(179956, 25)

In [47]:
###drop concord labels
combo_cat3 = combo_cat2[combo_cat2['Rep Owner'] != 'Concord']

In [48]:
### check row length
combo_cat3.shape

(179282, 25)

In [49]:
combo_cat3

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,eanfour,eanfive,Region,UPC_x,Transfer Price,Dealer Price,Net Unit Price,Currency,UPC_y,Rep Owner
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,0,0,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179974,2020,4,04/04/2020,www.amazon.de,UMG FR,Rudolf Serkin,The Originals-Brahms: Die Cellosonaten,1,Monthly Sales,DE,...,0,0,FR,28947937678,0.5038,4.89,4.45479,EUR,2.894794e+10,Deutsche Grammophon Intl
179975,2020,4,22/04/2020,www.amazon.co.uk,UMG UK,Val Doonican - His Special Years Box Set (Musi...,Very Best: His Special Years,1,Monthly Sales,UK,...,0,0,UK,28946640623,0.7700,7.55,6.19000,GBP,2.894664e+10,Decca
179976,2020,4,21/04/2020,www.amazon.de,UMG UK,Various Artists,Various Artists - La Phil Centenary Edition,1,Monthly Sales,DE,...,0,0,UK,28948361007,15.8700,59.47,48.77000,GBP,2.894836e+10,Deutsche Grammophon Intl
179977,2020,4,19/04/2020,www.amazon.co.uk,UMG IT,Wolfram Christ,Trout Quintet & Death And The Maiden (La Trota),1,Monthly Sales,UK,...,0,0,IT,28947842224,0.5800,3.00,2.44340,EUR,2.894784e+10,Decca


In [50]:
### check row count
combo_cat3.shape

(179282, 25)

In [51]:
#importing exception list

exception_list = pd.read_csv('G:\Amazon Repatriation May 2020\exception_list.csv', dtype={'UPC': np.int64})

In [52]:
#show exception list

exception_list.shape

(6166, 2)

In [53]:
##delete duplicates from exception list

exception_list.drop_duplicates(subset ="UPC", 
                     keep = False, inplace = True)

In [54]:
##show exception list
exception_list.shape

(5677, 2)

In [55]:
##only keep first label
exception_list2 = exception_list.drop_duplicates(subset='UPC', keep="first")

In [56]:
exception_list2.shape

(5677, 2)

In [57]:
#merging data with exception list to find artist exceptions

data_with_exception = pd.merge(combo_cat,exception_list[['UPC','Exception']],left_on="eanone"or"eantwo"or"eanthree"or"eanfour"or"eanfive", right_on=['UPC'], how='left')

In [58]:
##show data with exception

data_with_exception

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,Region,UPC_x,Transfer Price,Dealer Price,Net Unit Price,Currency,UPC_y,Rep Owner,UPC,Exception
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,NaN
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,NaN
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,NaN
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,NaN
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,DE,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179974,2020,4,04/04/2020,www.amazon.de,UMG FR,Rudolf Serkin,The Originals-Brahms: Die Cellosonaten,1,Monthly Sales,DE,...,FR,28947937678,0.5038,4.89,4.45479,EUR,2.894794e+10,Deutsche Grammophon Intl,2.894794e+10,Classical
179975,2020,4,22/04/2020,www.amazon.co.uk,UMG UK,Val Doonican - His Special Years Box Set (Musi...,Very Best: His Special Years,1,Monthly Sales,UK,...,UK,28946640623,0.7700,7.55,6.19000,GBP,2.894664e+10,Decca,NaN,NaN
179976,2020,4,21/04/2020,www.amazon.de,UMG UK,Various Artists,Various Artists - La Phil Centenary Edition,1,Monthly Sales,DE,...,UK,28948361007,15.8700,59.47,48.77000,GBP,2.894836e+10,Deutsche Grammophon Intl,NaN,NaN
179977,2020,4,19/04/2020,www.amazon.co.uk,UMG IT,Wolfram Christ,Trout Quintet & Death And The Maiden (La Trota),1,Monthly Sales,UK,...,IT,28947842224,0.5800,3.00,2.44340,EUR,2.894784e+10,Decca,NaN,NaN


In [59]:
##assigning repat percentage, changing Nan to other 50%
data_with_exception['Exception'] = data_with_exception['Exception'].fillna(.50)

In [60]:
##assigning repat percentage, classical 60%, artist exception 40%, other 50%
 
data_with_exception.loc[data_with_exception['Exception'] == .5, 'repat_%'] = .5
data_with_exception.loc[data_with_exception['Exception'] == 'Artist ICLA', 'repat_%'] = .4
data_with_exception.loc[data_with_exception['Exception'] == 'Classical', 'repat_%'] = .6

In [61]:
##show data

data_with_exception

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,UPC_x,Transfer Price,Dealer Price,Net Unit Price,Currency,UPC_y,Rep Owner,UPC,Exception,repat_%
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,42282998620,0.6400,4.85,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179974,2020,4,04/04/2020,www.amazon.de,UMG FR,Rudolf Serkin,The Originals-Brahms: Die Cellosonaten,1,Monthly Sales,DE,...,28947937678,0.5038,4.89,4.45479,EUR,2.894794e+10,Deutsche Grammophon Intl,2.894794e+10,Classical,0.6
179975,2020,4,22/04/2020,www.amazon.co.uk,UMG UK,Val Doonican - His Special Years Box Set (Musi...,Very Best: His Special Years,1,Monthly Sales,UK,...,28946640623,0.7700,7.55,6.19000,GBP,2.894664e+10,Decca,NaN,0.5,0.5
179976,2020,4,21/04/2020,www.amazon.de,UMG UK,Various Artists,Various Artists - La Phil Centenary Edition,1,Monthly Sales,DE,...,28948361007,15.8700,59.47,48.77000,GBP,2.894836e+10,Deutsche Grammophon Intl,NaN,0.5,0.5
179977,2020,4,19/04/2020,www.amazon.co.uk,UMG IT,Wolfram Christ,Trout Quintet & Death And The Maiden (La Trota),1,Monthly Sales,UK,...,28947842224,0.5800,3.00,2.44340,EUR,2.894784e+10,Decca,NaN,0.5,0.5


In [62]:
##change repat % column to float for equations
data_with_exception['repat_%'] = data_with_exception['repat_%'].astype(float)

In [63]:

##Repatriation equation bracket net unit price times repat% price brackets minus transfer price

data_with_exception['Repatriation'] = (data_with_exception['Net Unit Price'] * data_with_exception['repat_%']) - data_with_exception['Transfer Price']

In [64]:
##show data
data_with_exception.head()

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,Transfer Price,Dealer Price,Net Unit Price,Currency,UPC_y,Rep Owner,UPC,Exception,repat_%,Repatriation
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0.64,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,0.64,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0.64,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,0.64,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,0.64,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605


In [65]:
##Repatriation percentage equation  bracket repatiation divded by dealer price bracket times one hundred

data_with_exception['Repatriation_Percentage_%'] = (data_with_exception['Repatriation'] / data_with_exception['Dealer Price']) *100

In [66]:
##show data

data_with_exception.head()

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,Dealer Price,Net Unit Price,Currency,UPC_y,Rep Owner,UPC,Exception,repat_%,Repatriation,Repatriation_Percentage_%
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784


In [67]:
##total repatriation percentage equation stock movements times repatriation 

data_with_exception['Total_Repatriation'] = data_with_exception['unpacked_units'] * data_with_exception['Repatriation']

In [68]:
##show data

data_with_exception.head()

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,Net Unit Price,Currency,UPC_y,Rep Owner,UPC,Exception,repat_%,Repatriation,Repatriation_Percentage_%,Total_Repatriation
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784,3.210
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784,3.210
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784,1.605
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784,1.605
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,4.49,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605,33.092784,4.815


In [69]:
#removing any rows with negative Total_Repatriation total 

data_with_exception.drop(data_with_exception.index[data_with_exception['Total_Repatriation'] < 0], inplace = True)

In [70]:
##show data

data_with_exception.shape

(178551, 31)

In [71]:
##remove rows with any NA for Total_Repatriation

data_with_exception = data_with_exception[data_with_exception['Total_Repatriation'].notna()]

In [72]:
data_with_exception.shape

(178551, 31)

In [73]:
##show final data

data_with_exception

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report Type,inventory_owner_group_combined,...,Net Unit Price,Currency,UPC_y,Rep Owner,UPC,Exception,repat_%,Repatriation,Repatriation_Percentage_%,Total_Repatriation
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605000,33.092784,3.210000
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605000,33.092784,3.210000
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605000,33.092784,1.605000
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605000,33.092784,1.605000
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,4.49000,EUR,4.228300e+10,Polydor UK,NaN,0.5,0.5,1.605000,33.092784,4.815000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179974,2020,4,04/04/2020,www.amazon.de,UMG FR,Rudolf Serkin,The Originals-Brahms: Die Cellosonaten,1,Monthly Sales,DE,...,4.45479,EUR,2.894794e+10,Deutsche Grammophon Intl,2.894794e+10,Classical,0.6,2.169074,44.357342,2.169074
179975,2020,4,22/04/2020,www.amazon.co.uk,UMG UK,Val Doonican - His Special Years Box Set (Musi...,Very Best: His Special Years,1,Monthly Sales,UK,...,6.19000,GBP,2.894664e+10,Decca,NaN,0.5,0.5,2.325000,30.794702,2.325000
179976,2020,4,21/04/2020,www.amazon.de,UMG UK,Various Artists,Various Artists - La Phil Centenary Edition,1,Monthly Sales,DE,...,48.77000,GBP,2.894836e+10,Deutsche Grammophon Intl,NaN,0.5,0.5,8.515000,14.318144,8.515000
179977,2020,4,19/04/2020,www.amazon.co.uk,UMG IT,Wolfram Christ,Trout Quintet & Death And The Maiden (La Trota),1,Monthly Sales,UK,...,2.44340,EUR,2.894784e+10,Decca,NaN,0.5,0.5,0.641700,21.390000,0.641700


In [74]:
data_with_exception.shape

(178551, 31)

In [87]:
##remove rows with any NA for inventory owner group

data_with_exception = data_with_exception[data_with_exception['inventory_owner_group_combined']!= 0]

In [88]:
del data_with_exception['UPC']

KeyError: 'UPC'

In [89]:
data_with_exception.head()

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report_Type,inventory_owner_group_combined,...,Dealer_Price,Net_Unit_Price,Currency,UPC_y,Rep_Owner,Exception,repat_percentage,Repatriation,Repatriation_Percentage_Final,Total_Repatriation
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605,33.092784,3.210
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605,33.092784,3.210
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605,33.092784,1.605
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605,33.092784,1.605
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,4.85,4.49,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605,33.092784,4.815


In [90]:
data_with_exception

,calendar_year,calendar_month_of_year,received_day,inventory_owner_group,vendor_group,artist_primary,item_name,unpacked_units,Report_Type,inventory_owner_group_combined,...,Dealer_Price,Net_Unit_Price,Currency,UPC_y,Rep_Owner,Exception,repat_percentage,Repatriation,Repatriation_Percentage_Final,Total_Repatriation
0,2020,4,28/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.85,4.49000,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605000,33.092784,3.210000
1,2020,4,24/04/2020,DE,UMG DE,Magnum,Vigilante,2,Gross Shipments,DE,...,4.85,4.49000,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605000,33.092784,3.210000
2,2020,4,30/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.85,4.49000,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605000,33.092784,1.605000
3,2020,4,03/04/2020,DE,UMG DE,Magnum,Vigilante,1,Gross Shipments,DE,...,4.85,4.49000,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605000,33.092784,1.605000
4,2020,4,27/04/2020,DE,UMG DE,Magnum,Vigilante,3,Gross Shipments,DE,...,4.85,4.49000,EUR,4.228300e+10,Polydor UK,0.5,0.5,1.605000,33.092784,4.815000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179974,2020,4,04/04/2020,www.amazon.de,UMG FR,Rudolf Serkin,The Originals-Brahms: Die Cellosonaten,1,Monthly Sales,DE,...,4.89,4.45479,EUR,2.894794e+10,Deutsche Grammophon Intl,Classical,0.6,2.169074,44.357342,2.169074
179975,2020,4,22/04/2020,www.amazon.co.uk,UMG UK,Val Doonican - His Special Years Box Set (Musi...,Very Best: His Special Years,1,Monthly Sales,UK,...,7.55,6.19000,GBP,2.894664e+10,Decca,0.5,0.5,2.325000,30.794702,2.325000
179976,2020,4,21/04/2020,www.amazon.de,UMG UK,Various Artists,Various Artists - La Phil Centenary Edition,1,Monthly Sales,DE,...,59.47,48.77000,GBP,2.894836e+10,Deutsche Grammophon Intl,0.5,0.5,8.515000,14.318144,8.515000
179977,2020,4,19/04/2020,www.amazon.co.uk,UMG IT,Wolfram Christ,Trout Quintet & Death And The Maiden (La Trota),1,Monthly Sales,UK,...,3.00,2.44340,EUR,2.894784e+10,Decca,0.5,0.5,0.641700,21.390000,0.641700


In [91]:
data_with_exception.shape

(177842, 30)

In [92]:
## rename columns for google big query upload 
data_with_exception = data_with_exception.rename(columns={'Report Type': 'Report_Type', 'Rep Owner': 'Rep_Owner', 'Transfer Price':'Transfer_Price','Dealer Price':'Dealer_Price','Net Unit Price':'Net_Unit_Price','Currency ':'Currency','repat_%':'repat_percentage','Repatriation_Percentage_%':'Repatriation_Percentage_Final'})

In [93]:
###check for duplicates rows
sum(data_with_exception.duplicated())

3850

In [85]:
data_with_exception.to_gbq('commercial_affairs_analytics.amazon_repatriation_sat_test7_2020', 'umg-uk')

1it [00:43, 43.94s/it]
